# Классификация рукописных цифр при помощи метода KNN

In [ ]:
# База
import numpy as np
import pandas as pd
# Модели и метрики
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
# Визуализация
import plotly.express as px

In [ ]:
# Важные константы
# Для воспроизводимости
SEED = 202212
# Размер изображений
SHAPE = (28, 28)

## Загружаем данные

> **Источник данных:**
>
> https://www.kaggle.com/datasets/oddrationale/mnist-in-csv

In [ ]:
# Датасет уже разделен на две части,
# но мы потом поделим сами
df = pd.concat(
    [
        pd.read_csv('data/mnist_train.csv'),
        pd.read_csv('data/mnist_test.csv'),
    ], 
    ignore_index=True
)

In [ ]:
# Экономия памяти
df = df.astype(np.int16)

In [ ]:
# Достаточно взять 7000 примеров, или 10%
# Для ускорения работы
df = df.sample(frac=0.10, random_state=SEED, ignore_index=True)

In [ ]:
# Отделяем целевую переменную
X = df.drop(columns=['label'])
y = df['label']

## Посмотрим на данные

In [ ]:
def show_image(array):
    """Показывает изображение"""
    matrix = array.reshape(SHAPE)
    
    fig = px.imshow(
        matrix, 
        color_continuous_scale=['white', 'black']
    )
    fig.update_layout(
        width=650, 
        height=650, 
        coloraxis_showscale=False
    )
    
    fig.show()

### Примеры изображений

In [ ]:
for i in range(4):
    print(f'Цифра: {y[i]}')
    print(f'Изображение:')
    show_image(X.values[i])

### Проверим сбалансированность классов

In [ ]:
y_distr = df['label'].value_counts(normalize=True).sort_index()

fig = px.pie(
    values=y_distr.values, 
    names=y_distr.index, 
    hole=0.50,
    color_discrete_sequence=px.colors.sequential.Magma,
    template='plotly_white'
)

fig.update_traces(
    textposition='inside', 
    textinfo='percent+label'
)

fig.show()

>Так как классы распределены достаточно равномерно, 
>
>для оценки качества можно использовать процент правильных ответов

### Посмотрим на усредненное изображение

In [ ]:
mean_image = X[y == 5].mean(axis=0).values.reshape(SHAPE)

show_image(mean_image)

## Обучение простой модели

In [ ]:
# Для выбора модели используем одно разделение
# Кросс-валидацию сделаем позже для лучшей модели

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    random_state=SEED,
    # Классы распределены равномерно, 
    # но лучше добавим стратификацию
    stratify=y
)

In [ ]:
def test_knn(n_neighbors, metric_func=accuracy_score):
        
    knn = KNeighborsClassifier(
        n_neighbors=n_neighbors
    
    )
    knn.fit(X_train, y_train)
        
    return metric_func(
        y_test,
        knn.predict(X_test)
    )

In [ ]:
# Поиск лучшего параметра "число соседей"
k_search = {
    i: test_knn(n_neighbors=i)
    for i in range(1, 11)
}

In [ ]:
def _create_figure(search_dict, name, line_color):
    fig = px.line(
        x=list(search_dict.keys()), 
        y=list(search_dict.values()), 
        template='plotly_white'
    )
    fig.update_traces(
        line_color=line_color,
        name=name,
        showlegend=True,
        mode='lines+markers',
    )
    return fig

In [ ]:
fig = _create_figure(k_search, 'Чистый KNN', 'black')
fig.update_layout(
    xaxis_title='Число соседей',
    yaxis_title='Точность модели (на тесте)'
)
fig.show()

## Добавим метод главных компонент

>Проверим, улучшит ли метод главных компонент KNN
>
>Идея: PCA поможет выделить более ценные признаки

In [ ]:
# Будем использовать меньше 10% информации
100 * np.array([25, 50, 75]) / (28*28)

In [ ]:
# Для удобства напишем свой класс

class KNN_with_PCA:
    
    def __init__(self, n_components, n_neighbors):
        self.pca = PCA(n_components=n_components, random_state=SEED)
        self.knn = KNeighborsClassifier(n_neighbors=n_neighbors)
    
    def fit(self, X_train, y_train):
        self.pca.fit(X_train)
        X_pca = self.pca.transform(X_train)
        
        self.knn.fit(X_pca, y_train)
        
    def predict(self, X_test):
        X_pca = self.pca.transform(X_test)
        return self.knn.predict(X_pca)

In [ ]:
def test_pca_knn(n_components, n_neighbors, metric_func=accuracy_score):
    
    model = KNN_with_PCA(n_components=n_components, n_neighbors=n_neighbors)
    model.fit(X_train, y_train)
    
    return metric_func(
        y_test,
        model.predict(X_test)
    )

In [ ]:
search_dicts = {
    n_components: {
        i: test_pca_knn(n_neighbors=i, n_components=n_components)
        for i in range(1, 11)
    }
    # Проверим 25/50/75 компонент
    for n_components in [25, 50, 75]
}

### Сравнение чистого KNN и PCA+KNN

In [ ]:
fig = _create_figure(k_search, name='Чистый KNN', line_color='black')

colors = {
    25: 'orange',
    50: 'green',
    75: 'darkblue'
}

for i in [25, 50, 75]:
    fig_this = _create_figure(search_dicts[i], name=f'PCA ({i}) + KNN', line_color=colors[i])
    fig.add_traces(fig_this.data)

fig.update_traces(
    showlegend=True,
    mode='lines+markers',
)

fig.update_layout(
    template='plotly_white',
    xaxis_title='Число соседей',
    yaxis_title='Точность модели (на тесте)',
)

fig.show()

>**PCA+KNN** уверенно обходит обычный KNN по точности
>
>Интересно, что лучше всего подходит 1 ближайший сосед

### Посмотрим матрицу ошибок

In [ ]:
conf_matrix = test_pca_knn(
    n_components=50, 
    n_neighbors=1, 
    metric_func=confusion_matrix
)

In [ ]:
fig = px.imshow(
    conf_matrix,
    text_auto=True,
    color_continuous_scale='greys'
)

fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)

fig.show()

>Матрица ошибок также выглядит хорошо

### Кросс-валидация

In [ ]:
# Выбранная модель
chosen_model = KNN_with_PCA(
    n_components=50, 
    n_neighbors=1
)

splitter = StratifiedKFold(
    n_splits=5, 
    shuffle=True, 
    random_state=SEED
)
kfold = enumerate(splitter.split(X, y))

cross_validation_accuracy = {}
for i, (train_index, test_index) in kfold:
    
    chosen_model.fit(X.iloc[train_index], y.iloc[train_index])
    
    cross_validation_accuracy[i] = accuracy_score(
        y.iloc[test_index],
        chosen_model.predict(X.iloc[test_index])
    )

In [ ]:
fig = _create_figure(
    cross_validation_accuracy, 
    name='Кросс-валидация', 
    line_color='green'
)
fig.update_traces(
    showlegend=True,
    mode='lines+markers',
    line_dash='dash',
)
fig.update_layout(
    xaxis_title='Номер итерации',
    yaxis_title='Точность модели (на тесте)'
)
fig.show()

>Хотя есть различия по итерациям, в целом модель работает достаточно стабильно